# Exploring Hacker News Posts

For this project, we will be exploring 'Ask HN' and 'Show HN' posts on Hacker News to see if:

- Ask HN or Show HN posts receive more comments on average
- Posts created at a certain time receive more comments on average


The data is taken from this [link](https://www.kaggle.com/hacker-news/hacker-news-posts), but it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that didn't receive any comments and then randomly sampling from the remaining submissions. 

To start with, we'll read in the data as a list of lists without the header row

In [32]:
import pandas as pd

# Create a dataframe from csv
df = pd.read_csv('hacker_news.csv')
# User list comprehension to create a list of lists from Dataframe rows
hn = [list(row) for row in df.values]
# Print list of lists i.e. rows
print(hn[0:5])

headers = list(df.columns.values)
print(headers)

[[12224879, 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', 386, 52, 'ne0phyte', '8/4/2016 11:52'], [10975351, 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', 39, 10, 'josep2', '1/26/2016 19:30'], [11964716, "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', 2, 1, 'vezycash', '6/23/2016 22:20'], [11919867, 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', 3, 1, 'hswarna', '6/17/2016 0:01'], [10301696, 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', 8, 2, 'walterbell', '9/30/2015 4:12']]
['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


Note: How do I check for duplicates or errors?

Now that we have the data, we'll split the articles into those that start with 'Ask HN', 'Show HN' and others, taking into account the possiblity that the phrases may appear with both upper and lower case letters.

In [33]:
ask_posts = []
show_posts = []
other_posts =[]

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
    
print(len(show_posts))
print(len(ask_posts))
print(len(other_posts))
print(len(hn) == len(show_posts) + len(ask_posts) + len(other_posts))

1162
1744
17194
True


In [34]:
print(type(len(ask_posts)))

<class 'int'>


Now that we've split the different types of posts into separate lists of lists, we'll see if 'Ask HN' or 'Show HN' posts receive more comments on average. To do this, we will need to find the total number of comments for each type of post, and divide that by the total number of that type of post.

In [35]:
# Finding the average number of comments per 'Ask HN' post
total_ask_comments = 0

for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments
    
total_ask_posts = len(ask_posts)

avg_ask_comments = total_ask_comments / total_ask_posts
print(avg_ask_comments)

14.038417431192661


In [36]:
# Finding the average number of comments per 'Show HN' post
total_show_comments = 0

for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments
    
total_show_posts = len(show_posts)

avg_show_comments = total_show_comments / total_show_posts
print(avg_show_comments)

10.31669535283993


We can see that 'Ask HN' posts receive more comments on average than 'Show HN' posts: 14.0 compared to 10.3 comments per post within the data set.

Now that we've determined that 'Ask HN' posts receie more comments on average, we want to see if 'Ask HN' posts created at a certain time are more likely to attract comments. We will do this by:

1. Calculating the number of ask posts created in each hour of the day, along with the number of comments received.
2. Calculating the average number of comments ask posts receive by hour created.

We'll start by calculating the number of ask posts and comments by hour created.

In [37]:
import datetime as dt

In [38]:
result_list = []

for post in ask_posts:
    num_comments = int(post[4])
    created_at = post[6]
    info = [created_at, num_comments]
    result_list.append(info)
    
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    created_dt = dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M")
    hour = created_dt.strftime("%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    elif hour in counts_by_hour:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]
        
print(counts_by_hour)
print('\n')
print(comments_by_hour)

{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}


{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


Now that we have 2 dictionaries, one with the number of posts per hour, and another with the number of comments per hour, we can calculate the average number of comments for posts created during each hour of the day.

In [39]:
avg_by_hour = []

for hour in counts_by_hour:
    comments = int(comments_by_hour[hour])
    avg_by_hour.append([hour, (comments/int(counts_by_hour[hour]))])

print(avg_by_hour)

[['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


We now have the results we need, but this format makes it difficult to identify the hours with the highest values. So we'll sort the list of lists and print the five highest values in a format that's easier to read.

In [40]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swapped_row = [row[1], row[0]]
    swap_avg_by_hour.append(swapped_row)

print(swap_avg_by_hour)

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]


In [45]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print("Top 5 Hours for Ask Posts Comments")
for row in sorted_swap[0:5]:
    hour = dt.datetime.strptime(row[1],"%H")
    hour_string = hour.strftime("%H:%M")
    avg_comments = str(round(float(row[0]), 2))
    print(hour_string+": "+avg_comments+" average comments per post")

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.8 average comments per post
21:00: 16.01 average comments per post


The date and time that the posts are made are in the Eastern Time zone in the US. This suggests that the best time to create Ask HN posts to increase your chances of receiving comments is during the following hours in the GMT+8 timezone: 3-4am, 2-3pm, 9-10am, 4-5am and 9-10am. 

Additional analysis to consider:
    
- Determine if show or ask posts receive more points on average.
- Determine if posts created at a certain time are more likely to receive more points.
- Compare your results to the average number of comments and points other posts receive.